In [1]:
library(Seurat)
library(tidyverse)

Attaching SeuratObject

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.3.6      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.1      ✔ stringr 1.4.1 
✔ readr   2.1.2      ✔ forcats 0.5.2 
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [3]:
obj = qs::qread('full_obj_sct.qs')
obj

An object of class Seurat 
59498 features across 144643 samples within 3 assays 
Active assay: SCT (27177 features, 3000 variable features)
 2 other assays present: RNA, integrated
 2 dimensional reductions calculated: pca, umap

In [12]:
obj %>%
`[[` %>%
mutate(group = paste0(treatment, '.', strain, '.', time)) %>%
select(hash.mcl.ID, group) %>%
distinct %>%
group_by(group) %>%
summarise(n = n()) %>% print

# A tibble: 6 × 2
  group                 n
  <chr>             <int>
1 FGF1.obob.Day14       7
2 FGF1.obob.Day5        7
3 Veh_PF.BL6.Day14      4
4 Veh_PF.BL6.Day5       4
5 Veh_PF.obob.Day14     7
6 Veh_PF.obob.Day5      7


In [14]:
all_restored_summary = qs::qread('01_milo/_targets/objects/all_restored_summary')
all_restored_summary %>% head

labels,n_total,n_fgf1,n_fgf1.pos,n_fgf1.neg,prop_fgf1.pos,n_BL6,n_restored,n_restored.pos,n_restored.neg,prop_restored_BL6,prop_restored_FGF1,comparison
<chr>,<int>,<int>,<int>,<int>,<dbl>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<chr>
b1_Tanycytes,224,18,10,8,0.5555556,13,0,0,0,0.000000000,0.0,all.obob14v14__v__all.obob14vBL6
NG2_OPC2,106,6,4,2,0.6666667,44,0,0,0,0.000000000,0.0,all.obob14v14__v__all.obob14vBL6
b2_Tany,80,3,3,NA,1.0000000,1,0,0,0,0.000000000,0.0,all.obob14v14__v__all.obob14vBL6
Microglia,51,2,1,1,0.5000000,6,1,0,1,0.166666667,0.5,all.obob14v14__v__all.obob14vBL6
MOL,210,2,2,NA,1.0000000,134,1,1,0,0.007462687,0.5,all.obob14v14__v__all.obob14vBL6
Astrocytes,636,1,NA,1,NA,397,0,0,0,0.000000000,0.0,all.obob14v14__v__all.obob14vBL6


In [15]:
all_restored_summary %>%
select(comparison) %>%
distinct

comparison
<chr>
all.obob14v14__v__all.obob14vBL6
all.obob14v14__v__all.obob5vBL6
all.obob14v14__v__all.obobBL6
all.obob14v14__v__all.obobBL6d14
all.obob14v14__v__all.obobBL6d5
all.obob5v5__v__all.obob14vBL6
all.obob5v5__v__all.obob5vBL6
all.obob5v5__v__all.obobBL6
all.obob5v5__v__all.obobBL6d14


In [26]:
all_restored_summary %>%
filter(comparison == 'all.obob5v5__v__all.obobBL6') %>%
select(labels, n_total, n_fgf1) %>%
filter(n_fgf1 > 0) %>%
print(n=50)

# A tibble: 29 × 3
   labels                     n_total n_fgf1
   <chr>                        <int>  <int>
 1 Astrocytes                     636    230
 2 Htr3b                          238     77
 3 MOL                            210     75
 4 b1_Tanycytes                   224     72
 5 a1_Tany__Astrocytes            251     56
 6 Trh_Lef1                       208     53
 7 NG2_OPC2                       106     48
 8 a2_Tanycytes                   178     47
 9 Agrp                           318     30
10 Sst_Unc13c__Agrp               208     24
11 Sst_Pthlh.sc23                 145     20
12 NFOL                            21     19
13 Microglia                       51     17
14 Pomc_Glipr1__Slc17a6_Trhr      233     13
15 Tmem215                        281     12
16 a2_Tanycytes__b1_Tanycytes     108     11
17 Ghrh                           139      8
18 Kisspeptin                     189      7
19 Rgs16_Dlx1                      95      7
20 Unassigned1.sc17.sc28          27

In [27]:
all_restored_summary %>%
filter(comparison == 'all.obob14v14__v__all.obobBL6') %>%
select(labels, n_total, n_fgf1) %>%
filter(n_fgf1 > 0) %>%
print(n=50)

# A tibble: 6 × 3
  labels       n_total n_fgf1
  <chr>          <int>  <int>
1 b1_Tanycytes     224     18
2 NG2_OPC2         106      6
3 b2_Tany           80      3
4 Microglia         51      2
5 MOL              210      2
6 Astrocytes       636      1


In [31]:
all_restored_summary %>%
filter(comparison == 'all.obob5v5__v__all.obobBL6') %>%
select(labels, n_total, n_restored) %>%
filter(n_restored > 0) %>%
mutate(prop = n_restored/n_total) %>%
arrange(desc(prop)) %>%
print(n=50)

# A tibble: 21 × 4
   labels                     n_total n_restored    prop
   <chr>                        <int>      <int>   <dbl>
 1 MOL                            210         62 0.295  
 2 Astrocytes                     636        157 0.247  
 3 NG2_OPC2                       106         24 0.226  
 4 NFOL                            21          3 0.143  
 5 a1_Tany__Astrocytes            251         29 0.116  
 6 Sst_Unc13c__Agrp               208         15 0.0721 
 7 Htr3b                          238         17 0.0714 
 8 Agrp                           318         22 0.0692 
 9 Ghrh                           139          6 0.0432 
10 Pomc_Glipr1__Slc17a6_Trhr      233         10 0.0429 
11 Microglia                       51          2 0.0392 
12 Kisspeptin                     189          7 0.0370 
13 Tmem215                        281         10 0.0356 
14 b1_Tanycytes                   224          7 0.0312 
15 a2_Tanycytes                   178          5 0.0281 
16 Rgs16_Dlx

In [29]:
all_restored_summary %>%
filter(comparison == 'all.obob14v14__v__all.obobBL6') %>%
select(labels, n_total, n_restored) %>%
filter(n_restored > 0) %>%
print(n=50)

# A tibble: 3 × 3
  labels       n_total n_restored
  <chr>          <int>      <int>
1 b1_Tanycytes     224          1
2 Microglia         51          1
3 MOL              210          1
